<a href="https://colab.research.google.com/github/tirthpatel1111/Tops/blob/main/Nueral%20network%20and%20deep%20learning/Assignment/customer_segmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle # creating .kaggle folder to store kaggle file
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d thedevastator/predicting-credit-card-customer-attrition-with-m

Dataset URL: https://www.kaggle.com/datasets/thedevastator/predicting-credit-card-customer-attrition-with-m
License(s): CC0-1.0
  0% 0.00/379k [00:00<?, ?B/s]
100% 379k/379k [00:00<00:00, 490MB/s]


In [ ]:
!unzip /content/predicting-credit-card-customer-attrition-with-m.zip

Archive:  /content/predicting-credit-card-customer-attrition-with-m.zip
  inflating: BankChurners.csv        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df=pd.read_csv("/content/predicting-credit-card-customer-attrition-with-m.zip")

In [ ]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [ ]:
df.dtypes

,0
CLIENTNUM,int64
Attrition_Flag,object
Customer_Age,int64
Gender,object
Dependent_count,int64
Education_Level,object
Marital_Status,object
Income_Category,object
Card_Category,object
Months_on_book,int64


In [ ]:
num=[]
cat=[]
for i in df.columns:
  if df[i].dtype=="object":
    cat.append(i)
  else:
    num.append(i)

In [ ]:
print(cat)

['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']


In [ ]:
label_cat=['Attrition_Flag', 'Gender','Marital_Status']

In [ ]:
ordinal_cat=['Education_Level','Income_Category', 'Card_Category']

In [ ]:
df['Card_Category'].value_counts()

,count
Card_Category,
Blue,9436
Silver,555
Gold,116
Platinum,20


In [ ]:
customer_cat = [
    ["Unknown","Uneducated", "High School", "College", "Graduate", "Post-Graduate", "Doctorate"],  # Education_Level
    ["Less than $40K", "$40K - $60K", "$60K - $80K", "$80K - $120K", "$120K +", "Unknown"],  # Income_Category
    ["Blue", "Silver", "Gold", "Platinum"]  # Card_Category
]

In [ ]:
df.Income_Category.unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

In [ ]:
label=LabelEncoder()
Attrition_Flag=label.fit_transform(df['Attrition_Flag'])


In [ ]:
label=LabelEncoder()
Gender=label.fit_transform(df['Gender'])

In [ ]:
Marital_Status=label.fit_transform(df['Marital_Status'])

In [ ]:
preprocess=ColumnTransformer(
    transformers=[
        ("num",MinMaxScaler(),num),
        ("cat",OrdinalEncoder(categories=customer_cat),ordinal_cat),

        # ("nummin",MinMaxScaler(),ordinal_cat),
    ]
)

In [ ]:
process=Pipeline([
    ("preprocess",preprocess)
])

In [ ]:
df_transformed = process.fit_transform(df)


In [ ]:
feature_names = process.get_feature_names_out()

df_transformed = pd.DataFrame(df_transformed, columns=feature_names)


In [ ]:
df_transformed.head()

,num__CLIENTNUM,num__Customer_Age,num__Dependent_count,num__Months_on_book,num__Total_Relationship_Count,num__Months_Inactive_12_mon,num__Contacts_Count_12_mon,num__Credit_Limit,num__Total_Revolving_Bal,num__Avg_Open_To_Buy,num__Total_Amt_Chng_Q4_Q1,num__Total_Trans_Amt,num__Total_Trans_Ct,num__Total_Ct_Chng_Q4_Q1,num__Avg_Utilization_Ratio,num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,cat__Education_Level,cat__Income_Category,cat__Card_Category
0,0.504929,0.404255,0.6,0.604651,0.8,0.166667,0.500000,0.340190,0.308701,0.345116,0.392994,0.035273,0.248062,0.437534,0.061061,0.000086,0.99992,2.0,2.0,0.0
1,0.920398,0.489362,1.0,0.720930,1.0,0.166667,0.333333,0.206112,0.343266,0.214093,0.453636,0.043452,0.178295,1.000000,0.105105,0.000049,0.99995,4.0,0.0,0.0
2,0.049060,0.531915,0.6,0.534884,0.6,0.166667,0.000000,0.059850,0.000000,0.098948,0.763615,0.076611,0.077519,0.628164,0.000000,0.000013,0.99999,4.0,3.0,0.0
3,0.514130,0.297872,0.8,0.488372,0.4,0.666667,0.166667,0.056676,1.000000,0.022977,0.413600,0.036775,0.077519,0.628164,0.760761,0.000126,0.99988,2.0,0.0,0.0
4,0.008517,0.297872,0.6,0.186047,0.8,0.166667,0.000000,0.099091,0.000000,0.136557,0.640271,0.017025,0.139535,0.673129,0.000000,0.000014,0.99999,1.0,2.0,0.0


In [ ]:
df_transformed.shape

(10127, 20)

In [ ]:
norm=MinMaxScaler()
s=norm.fit_transform(df_transformed[['cat__Education_Level']])

In [ ]:
df_transformed['cat__Education_Level']=s


In [ ]:
Attrition_Flag

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
df['Attrition_Flag']=Attrition_Flag
df['Gender']=Gender
df['Marital_Status']=Marital_Status

In [ ]:
df['Attrition_Flag']

,Attrition_Flag
0,1
1,1
2,1
3,1
4,1
...,...
10122,1
10123,0
10124,0
10125,0


In [ ]:
df_transformed['Attrition_Flag']=Attrition_Flag
df_transformed['Gender']=Gender
df_transformed['Marital_Status']=Marital_Status

In [ ]:
df_transformed.head()

,num__CLIENTNUM,num__Customer_Age,num__Dependent_count,num__Months_on_book,num__Total_Relationship_Count,num__Months_Inactive_12_mon,num__Contacts_Count_12_mon,num__Credit_Limit,num__Total_Revolving_Bal,num__Avg_Open_To_Buy,...,num__Total_Ct_Chng_Q4_Q1,num__Avg_Utilization_Ratio,num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2,cat__Education_Level,cat__Income_Category,cat__Card_Category,Attrition_Flag,Gender,Marital_Status
0,0.504929,0.404255,0.6,0.604651,0.8,0.166667,0.500000,0.340190,0.308701,0.345116,...,0.437534,0.061061,0.000086,0.99992,0.333333,2.0,0.0,1,1,1
1,0.920398,0.489362,1.0,0.720930,1.0,0.166667,0.333333,0.206112,0.343266,0.214093,...,1.000000,0.105105,0.000049,0.99995,0.666667,0.0,0.0,1,0,2
2,0.049060,0.531915,0.6,0.534884,0.6,0.166667,0.000000,0.059850,0.000000,0.098948,...,0.628164,0.000000,0.000013,0.99999,0.666667,3.0,0.0,1,1,1
3,0.514130,0.297872,0.8,0.488372,0.4,0.666667,0.166667,0.056676,1.000000,0.022977,...,0.628164,0.760761,0.000126,0.99988,0.333333,0.0,0.0,1,0,3
4,0.008517,0.297872,0.6,0.186047,0.8,0.166667,0.000000,0.099091,0.000000,0.136557,...,0.673129,0.000000,0.000014,0.99999,0.166667,2.0,0.0,1,1,1


In [ ]:
df_transformed.columns

Index(['num__CLIENTNUM', 'num__Customer_Age', 'num__Dependent_count',
       'num__Months_on_book', 'num__Total_Relationship_Count',
       'num__Months_Inactive_12_mon', 'num__Contacts_Count_12_mon',
       'num__Credit_Limit', 'num__Total_Revolving_Bal', 'num__Avg_Open_To_Buy',
       'num__Total_Amt_Chng_Q4_Q1', 'num__Total_Trans_Amt',
       'num__Total_Trans_Ct', 'num__Total_Ct_Chng_Q4_Q1',
       'num__Avg_Utilization_Ratio',
       'num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'num__Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2',
       'cat__Education_Level', 'cat__Income_Category', 'cat__Card_Category',
       'Attrition_Flag', 'Gender', 'Marital_Status'],
      dtype='object')

In [ ]:
x=df_transformed.drop("Attrition_Flag",axis=1)
y=df_transformed["Attrition_Flag"]

In [ ]:
from re import X
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
y

,Attrition_Flag
0,1
1,1
2,1
3,1
4,1
...,...
10122,1
10123,0
10124,0
10125,0


In [ ]:


import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
if torch.cuda.is_available():
  device="cuda"
else:
  device="cpu"

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,x,y):
    self.x=torch.tensor(x.values,dtype=torch.float32)
    self.y=torch.tensor(y.values,dtype=torch.float32)
  def __getitem__(self,index):
    return self.x[index],self.y[index]
  def __len__(self):
    return len(self.x)

In [ ]:
train_dataset=CustomDataset(x_train,y_train)
test_dataset=CustomDataset(x_test,y_test)

In [ ]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)

In [ ]:
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
class ann(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.networks=nn.Sequential(
        nn.Linear(num_features,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,32),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(32,1),

        nn.Sigmoid()
    )

  def forward(self,x):
    return self.networks(x)


In [ ]:
epoch=20
lr=0.01

In [ ]:
model=ann(df_transformed.shape[1]-1)
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=lr)

In [ ]:
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=10, gamma=0.01)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# scheduler = ReduceLROnPlateau(
#     optimizer,        # your optimizer (e.g., Adam, SGD)
#     mode='min',       # 'min' if you monitor loss, 'max' if you monitor accuracy
#     factor=0.1,       # multiply LR by this factor (e.g., new_lr = old_lr * 0.2)
#     patience=0,       # epochs with no improvement after which LR is reduced
#       # prints updates
#     min_lr=1e-6,      # lower bound on LR
#     threshold=1e-4    # equivalent to min_delta in Keras
# )

In [ ]:
import copy
import torch

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, restore_best_weights=True, verbose=False):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.verbose = verbose

        self.best_loss = float("inf")
        self.best_weights = None
        self.wait = 0
        self.stopped_epoch = 0

    def __call__(self, current_loss, model, epoch):
        if current_loss < self.best_loss - self.min_delta:
            self.best_loss = current_loss
            self.wait = 0
            if self.restore_best_weights:
                self.best_weights = copy.deepcopy(model.state_dict())
            if self.verbose:
                print(f"Epoch {epoch}: 🔥 Training loss improved to {current_loss:.6f}")
        else:
            self.wait += 1
            if self.verbose:
                print(f"Epoch {epoch}: ⏳ No improvement. Wait = {self.wait}/{self.patience}")
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                if self.restore_best_weights and self.best_weights is not None:
                    model.load_state_dict(self.best_weights)
                    if self.verbose:
                        print("Restored best model weights.")
                return True  # signal to stop
        return False


In [ ]:
early_stop = EarlyStopping(patience=15, min_delta=0.0001, verbose=True)

for e in range(epoch):
    model.train()
    total_epoch_loss = 0

    for batch_features, batch_label in train_loader:
        batch_features = batch_features.float().to(device)
        batch_label = batch_label.float().unsqueeze(1).to(device)

        outputs = model(batch_features)
        loss = criterion(outputs, batch_label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss / len(train_loader)
    print(f"Epoch {e+1}/{epoch} - Train Loss: {avg_loss:.6f}")

    # Scheduler step on train loss
    # scheduler.step(avg_loss)

    # Early stopping check
    if early_stop(avg_loss, model, e+1):
        print("⏹ Early stopping triggered.")
        break


Epoch 1/20 - Train Loss: 0.374023
Epoch 1: 🔥 Training loss improved to 0.374023
Epoch 2/20 - Train Loss: 0.374023
Epoch 2: ⏳ No improvement. Wait = 1/15
Epoch 3/20 - Train Loss: 0.374023
Epoch 3: ⏳ No improvement. Wait = 2/15
Epoch 4/20 - Train Loss: 0.374530
Epoch 4: ⏳ No improvement. Wait = 3/15
Epoch 5/20 - Train Loss: 0.374307
Epoch 5: ⏳ No improvement. Wait = 4/15
Epoch 6/20 - Train Loss: 0.374530
Epoch 6: ⏳ No improvement. Wait = 5/15
Epoch 7/20 - Train Loss: 0.374025
Epoch 7: ⏳ No improvement. Wait = 6/15
Epoch 8/20 - Train Loss: 0.374622
Epoch 8: ⏳ No improvement. Wait = 7/15
Epoch 9/20 - Train Loss: 0.374276
Epoch 9: ⏳ No improvement. Wait = 8/15
Epoch 10/20 - Train Loss: 0.374095
Epoch 10: ⏳ No improvement. Wait = 9/15
Epoch 11/20 - Train Loss: 0.374590
Epoch 11: ⏳ No improvement. Wait = 10/15
Epoch 12/20 - Train Loss: 0.374276
Epoch 12: ⏳ No improvement. Wait = 11/15
Epoch 13/20 - Train Loss: 0.374026
Epoch 13: ⏳ No improvement. Wait = 12/15
Epoch 14/20 - Train Loss: 0.37412

In [ ]:
# for e in tqdm(range(epoch)):
#   model.train()
#   total_epoch_loss=0
#   for batch_features,batch_label in train_loader:
#     batch_features=batch_features.float().to(device)
#     batch_label=batch_label.float().unsqueeze(1).to(device)
#     outputs=model(batch_features)
#     loss=criterion(outputs,batch_label)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     total_epoch_loss+=loss.item()
#   avg_loss=total_epoch_loss/len(train_loader)
#   print("epoch",e,"avg_loss",avg_loss)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 10%|█         | 1/10 [00:00<00:08,  1.07it/s]

epoch 0 avg_loss 0.37991727172859074


 20%|██        | 2/10 [00:01<00:06,  1.26it/s]

epoch 1 avg_loss 0.3746754818075285


 30%|███       | 3/10 [00:02<00:04,  1.42it/s]

...

epoch 7 avg_loss 0.3743363937993688


 90%|█████████ | 9/10 [00:05<00:00,  1.67it/s]

epoch 8 avg_loss 0.3740971103662581


100%|██████████| 10/10 [00:06<00:00,  1.59it/s]

epoch 9 avg_loss 0.3744041151183797


In [ ]:
x_test.shape

(2026, 22)

In [ ]:
model.eval()
accuracy_list = []
with torch.no_grad():
    for batch_feature, batch_label in test_loader:
        batch_feature = batch_feature.to(device)
        batch_label = batch_label.to(device)
        print("batch_label",batch_label)
        # get logits
        logits = model(batch_feature)

        # convert logits -> probabilities
        probs = torch.sigmoid(logits)
        print(probs)
        # apply threshold
        y_pred = (probs > 0.5).float()
        print("y_pred",y_pred)
        # make sure target has shape [batch,1]
        batch_accuracy = (y_pred == batch_label.view(-1,1)).float().mean()
        # print(batch_accuracy.item())
        accuracy_list.append(batch_accuracy.item())

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print("Overall accuracy:", overall_accuracy)


batch_label tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])
tensor([[0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.5000],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.5000],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.7311],
        [0.5000]])
y_pred tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
       